In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle



In [3]:
from tensorflow.keras.models import load_model
## Load the trained model
model = load_model('model.h5')

## Load the encoder and scaler

encoder = pickle.load(open('encoders.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))
ohe= pickle.load(open('one hot encoders.pkl', 'rb'))





d:\Udemy ML\projects\Deep Learning\ann\ann\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
## Example input data
# Define the keys and values
keys = ["CreditScore", "Geography", "Gender", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary", "Exited"]
values = [619, "France", "Female", 42, 2, 0, 1, 1, 1, 101348.88, 1]

# Create the dictionary
input_data = dict(zip(keys, values))

# Print the dictionary
print(input_data)


{'CreditScore': 619, 'Geography': 'France', 'Gender': 'Female', 'Age': 42, 'Tenure': 2, 'Balance': 0, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 101348.88, 'Exited': 1}


In [6]:
# one hot encode 'Geography'
geo_encoded= ohe.transform([[input_data['Geography']]]).toarray()
geo_encoded= pd.DataFrame(geo_encoded, columns= ohe.get_feature_names_out(['Geography']))
geo_encoded
   
   
    

d:\Udemy ML\projects\Deep Learning\ann\ann\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,101348.88,1


In [8]:
## Encode categorical features

input_df['Gender'] = encoder.transform(input_df['Gender'])

input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0,1,1,1,101348.88,1


In [10]:
## concatination one hot encoded 
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0,1,1,1,101348.88,1,1.0,0.0,0.0


In [12]:
input_df_scaled = scaler.transform(input_df.drop('Exited', axis=1))
input_df_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
# prediction

prediction = model.predict(input_df_scaled)

prediction_probability = prediction[0][0]

prediction_probability




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


0.33102548

In [15]:
if prediction_probability > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
